# Homework 2 ATED - Group 1

In [1]:
%pylab inline
import math
import time
import copy
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('dark_background')

Populating the interactive namespace from numpy and matplotlib


## HW2-1

##### (a) Estimate the market price of the $5\%$ call spread (i.e. with strikes ATM and $5\%$ OTM). What about the $5\%$ put spread?

In [2]:
# TODO

##### (b) If you were to price the spreads in the Black-Scholes model using a single volatility parameter $\sigma$, what value of $\sigma$ would match the theoretical price with the market price? Comment on your results.

In [2]:
# TODO

## HW2-2

##### (a) Using the numerical package of your choice, calibrate the parameters of the SVI model against the market-implied volatility data. Show a comparative graph of the SVI curve and the actual implied volatility data points.

In [2]:
# TODO

##### (b) Compute or estimate the price of an at-the-money digital call option paying off $\$1$ if in one year NDX is greater than its current spot level, and zero otherwise: (i) in the Black-Scholes model, (ii) using $\pm 1\%$ call spreads, (iii) using the smile-adjusted formula on page 19.

In [2]:
# TODO

##### (c) Graph the implied distribution corresponding to the SVI model calibration.

In [2]:
# TODO

##### (d) Use the implied distribution to compute the price of the following European exotic options, where $X_0$ is the current index level and $X_T$ is the final index level:

##### i. Digital call defined in question (b);

In [2]:
# TODO

##### ii. "Reverse convertible" paying off $\max\left(100\%, 100\% + p \times \frac{X_T- X_0}{X_0}\right)$ if $\frac{X_T}{X_0} > 75 \%$ and $\frac{X_T}{X_0}$ otherwise, where $p = 50\%$. Then solve for $p$ to get a price of $100\%$;

In [2]:
# TODO

##### iii. Option paying off $\max\left(0, \frac{X_T- X_0}{X_T}\right)$;


In [2]:
# TODO

##### iv. Log-contract paying off $-2\log\left(\frac{X_T}{X_0}\right)$. Price interpretation;

In [2]:
# TODO

## HW2-3

#### Find conditions on the SVI model parameters to satisfy Lee’s asymptotic bounds on p. $22$: 
$$\sigma^{\star^2} (k_F,T) \leq \beta T |\log{k_F}|,\, \beta \in [0,2]$$

In [2]:
# TODO

## HW2-4

#### (Problem $4.3$ p$.\ 56$ in textbook, with corrections): Consider an underlying stock $S$ currently trading at $S_0 = 100$ which does not pay any dividend. Assume the local volatility function is $\sigma_{loc} (t, S) = \frac{0.1 - 0.15 \times \log\left(\frac{S}{S_0}\right)}{\sqrt{t}}$, and that interest rates are zero.

##### (a) Produce the graph of the local volatility surface for spots $0$ to $200$ and maturities $0$ to $5$ years.

In [2]:
# TODO

##### (b) Write a Monte-Carlo algorithm to price the following $1$-year payoffs using $252$ time steps and e.g.\ $10,000$ paths:

##### i. "Capped quadratic" option: $\min\left(1,\frac{S_1^2}{S_0^2}\right)$;

In [2]:
# TODO

##### ii. Asian at-the-money-call: $\max\left(0, \frac{S_{0.25} + S_{0.5} + S_{0.75} + S_1}{4 \times S_0} - 1\right)$;

In [2]:
# TODO

##### iii. Barrier call: $\max(0, S_1 - S_0)$ if $S$ always traded above $80$ using $252$ daily observations, 0 otherwise;

In [2]:
# TODO

## HW2-5

#### The payoff of a $1$-year at-the-money call on the geometric average return of two non-dividend paying stocks $X, Y$ is given as: $$ f(X_T,Y_T) = \max\left(0, \sqrt{\frac{X_T Y_T}{X_0 Y_0}} - 1\right)$$ where $T = 1$ year and $X_t, Y_t$ are the respective underlying spot prices of $X, Y$ at any time $t$.

##### (a) Derive analytical formulas for the call value at any time $0 \leq t \leq T$ in the Black-Scholes model with constant correlation $\rho$ (cf.\ Section $6-4$ in the textbook, to be covered during Session $5$.)

In [2]:
# TODO

##### (b) Compute the value of the call using a $5\%$ interest rate, $20\%$ volatility for $X$, $30\%$ volatility for $Y$, and $\rho = 0.4$. Use finite differences to estimate the deltas, gammas and cross-gamma of the call.

In [2]:
# TODO

##### (c) You purchased the call on a $\$10,000,000$ notional. What actions would you take to delta-hedge your position? What would then be your instant $P\&L$ in the matrix of scenarios. Generally, graph your instant $P\&L$ against percent changes $x, y$ in underlying stock prices.

In [2]:
# TODO